In [2]:
import numpy as np
from scipy.interpolate import CubicSpline
from scipy.stats import norm

In [12]:
import numpy as np
import scipy.stats as sct
from scipy.interpolate import CubicSpline
from scipy.stats import norm

# Given market data
times = np.array([1/52, 2/52, 3/52, 1/12, 2/12, 3/12, 4/12, 5/12, 6/12, 9/12, 1, 1.5, 2, 3, 4, 5, 6, 7, 10, 15, 20, 25, 30])
atmf_vol = np.array([2.22, 2.2275, 2.325, 2.43, 2.58, 2.7075, 2.95, 3.0795, 3.1775, 3.4025, 35925, 3.8175, 3.96, 4.1625, 4.3825, 4.5325, 4.611, 4.665, 4.635, 4.5275, 4.527, 4.526, 4.5255])
rr_25 = np.array([0.78, 0.7975, 0.7725, 0.775, 0.765, 0.75, 0.7375, 0.727, 0.725, 0.695, 0.735, 0.715, 0.705, 0.7075, 0.71, 0.71, 0.6995, 0.6925, 0.695, 0.7, 0.4015, 0.403, 0.4035])
bf_25 = np.array([0.3, 0.3, 0.295, 0.3, 0.2975, 0.3025, 0.3, 0.2945, 0.2925, 0.3075, 0.315, 0.3275, 0.33, 0.365, 0.365, 0.3875, 0.396, 0.4025, 0.4, 0.6175, 0.436, 0.436, 0.4355])

# Convert volatilities to decimal
atmf_vol /= 100
rr_25 /= 100
bf_25 /= 100

# Interpolation functions
cs_atmf = CubicSpline(times, atmf_vol)
cs_rr_25 = CubicSpline(times, rr_25)
cs_bf_25 = CubicSpline(times, bf_25)

# Function to calculate Black-Scholes price
def black_scholes_fx_price(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * np.exp(-rf * T) * norm.cdf(d1) - K * np.exp(-rd * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-rd * T) * norm.cdf(-d2) - S * np.exp(-rf * T) * norm.cdf(-d1)
    return price

# Function to calculate vega
def black_scholes_vega(S, K, T, rd, rf, sigma):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * norm.pdf(d1)
    return vega

# Function to calculate vanna
def black_scholes_vanna(S, K, T, rd, rf, sigma):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    vega = black_scholes_vega(S, K, T, rd, rf, sigma)
    vanna = vega * (d2 / (S * sigma))
    return vanna

# Function to calculate volga
def black_scholes_volga(S, K, T, rd, rf, sigma):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    vega = black_scholes_vega(S, K, T, rd, rf, sigma)
    volga = vega * (d1 * d2 / sigma)
    return volga

# Function to calculate Vanna-Volga adjusted price
def vanna_volga_fx_price(S, K, T, rd, rf, sigma_ATM, sigma_RR, sigma_BF, option_type='call'):
    bs_price = black_scholes_fx_price(S, K, T, rd, rf, sigma_ATM, option_type)
    vega_atm = black_scholes_vega(S, K, T, rd, rf, sigma_ATM)
    vega_rr = black_scholes_vega(S, K, T, rd, rf, sigma_RR)
    vega_bf = black_scholes_vega(S, K, T, rd, rf, sigma_BF)
    vanna_rr = black_scholes_vanna(S, K, T, rd, rf, sigma_RR)
    volga_bf = black_scholes_volga(S, K, T, rd, rf, sigma_BF)
    
    # Solve for weights
    X = np.array([vega_atm, vanna_rr, volga_bf])
    A = np.array([[vega_atm, vega_rr, vega_bf],
                  [black_scholes_vanna(S, K, T, rd, rf, sigma_ATM), black_scholes_vanna(S, K, T, rd, rf, sigma_RR), black_scholes_vanna(S, K, T, rd, rf, sigma_BF)],
                  [black_scholes_volga(S, K, T, rd, rf, sigma_ATM), black_scholes_volga(S, K, T, rd, rf, sigma_RR), black_scholes_volga(S, K, T, rd, rf, sigma_BF)]])
    
    w = np.linalg.solve(A, X)
    
    # Calculate the adjustment
    market_prices = np.array([sigma_RR, sigma_BF, sigma_ATM])
    BS_prices = np.array([black_scholes_fx_price(S, K, T, rd, rf, vol, option_type) for vol in [sigma_RR, sigma_BF, sigma_ATM]])
    
    adjustments = (market_prices)
    VV_adjustment = np.dot(w, adjustments)
    
    p = 0.25  # Adjustment factor
    vv_price = bs_price + p * VV_adjustment
    
    return vv_price

# Example usage
S = 83.56  # Spot price
K = 84.12  # Strike price
T = 0.5    # Time to maturity in years
rd = 0.06481  # Domestic risk-free rate
rf = 0.05132  # Foreign risk-free rate
sigma_ATM = cs_atmf(T)  # Interpolated ATM volatility
sigma_RR = cs_rr_25(T)  # Interpolated 25 Delta RR volatility
sigma_BF = cs_bf_25(T)  # Interpolated 25 Delta BF volatility

price = vanna_volga_fx_price(S, K, T, rd, rf, sigma_ATM, sigma_RR, sigma_BF, option_type='call')
print(f"Vanna-Volga Adjusted Price: {price}")

# also calculate the BS price
bs_price = black_scholes_fx_price(S, K, T, rd, rf, sigma_ATM, option_type='call')
print(f"Black-Scholes Price: {bs_price}")


Vanna-Volga Adjusted Price: 0.7562442751469858
Black-Scholes Price: 0.7326549977755477


Vanna-Volga Adjusted Price: 76.5445171736724
Black-Scholes Price: 73.69325194101428


In [22]:

# Black-Scholes prices
price_atmf_call = black_scholes_fx(S, K, T, rd, rf, sigma_atmf, 'call')
price_10d_call = black_scholes_fx(S, K, T, rd, rf, sigma_10d, 'call')
price_25d_call = black_scholes_fx(S, K, T, rd, rf, sigma_25d, 'call')

price_atmf_put = black_scholes_fx(S, K, T, rd, rf, sigma_atmf, 'put')
price_10d_put = black_scholes_fx(S, K, T, rd, rf, sigma_10d, 'put')
price_25d_put = black_scholes_fx(S, K, T, rd, rf, sigma_25d, 'put')

# print the prices
print('ATMF call price: ', price_atmf_call)
print('10-delta call price: ', price_10d_call)
print('25-delta call price: ', price_25d_call)
print('ATMF put price: ', price_atmf_put)
print('10-delta put price: ', price_10d_put)
print('25-delta put price: ', price_25d_put)

ATMF call price:  7.836840895386928
10-delta call price:  9.726498246439157
25-delta call price:  8.593311148020142
ATMF put price:  7.360035162148328
10-delta put price:  9.249692513200579
25-delta put price:  8.116505414781557


In [23]:

# Vega values
vega_atmf = vega(S, K, T, rd, rf, sigma_atmf)
vega_10d = vega(S, K, T, rd, rf, sigma_10d)
vega_25d = vega(S, K, T, rd, rf, sigma_25d)

In [27]:
# Correct Risk Reversal (RR) and Butterfly (BF) values using 25-delta
value_rr = (price_25d_call - price_25d_put) - (price_atmf_call - price_atmf_put)
value_bf = (price_25d_call + price_25d_put) / 2 - (price_atmf_call + price_atmf_put) / 2

In [28]:
# Vanna and Volga ratios
exotic_vanna_ratio = vega_10d / vega_atmf
exotic_volga_ratio = vega_25d / vega_atmf

In [29]:

# Vanna and Volga costs
cost_of_vanna = exotic_vanna_ratio * value_rr
cost_of_volga = exotic_volga_ratio * value_bf

# Adjusted price
adjusted_price = price_atmf_call + cost_of_vanna + cost_of_volga

price_atmf_call, cost_of_vanna, cost_of_volga, adjusted_price

(7.836840895386928,
 -1.4172537159449139e-14,
 0.7557173613518778,
 8.59255825673879)